<a href="https://colab.research.google.com/github/JanetteFu/Message_Detect/blob/main/Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv('/content/sample_data/clean_tweets_sentiment140_32k.csv')

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data = shuffle(data)
data['sentiment'] = data['sentiment'].replace(4,'1')
data['sentiment'] = data['sentiment'].replace(0,'0')
data.head()

,Unnamed: 0,text,hashtag,emojis,sentiment
328,328,not yet unfortunately another few weeks i ve been told how are you i ve been a little bit busy with my latest project,[],[],0
736,736,still doing my homework,[],[],0
19841,1587841,you got an course like that in your degree never heard of it,[],[],1
24409,1592409,you stupid girl hahah do not be negative i want it too but i have to wait we can be tortured together okay,[],[],1
6543,6543,more snow this morning here in the adirondacks it s covering my crocus that were coming up,[],[],0


In [ ]:
import re, nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
#from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()



def normalizer(tweet):
  tokens = nltk.word_tokenize(tweet)
  #filtered_result = list(filter(lambda l: l not in stop_words, tokens))
  lemmas = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
  return lemmas

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pd.set_option('display.max_colwidth', -1)
data['text'] = data.text.apply(str)
data['normalized_tweet'] = data.text.apply(normalizer)
data[['text', 'normalized_tweet']].head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,normalized_tweet
328,not yet unfortunately another few weeks i ve been told how are you i ve been a little bit busy with my latest project,"[not, yet, unfortunately, another, few, week, i, ve, been, told, how, are, you, i, ve, been, a, little, bit, busy, with, my, latest, project]"
736,still doing my homework,"[still, doing, my, homework]"
19841,you got an course like that in your degree never heard of it,"[you, got, an, course, like, that, in, your, degree, never, heard, of, it]"
24409,you stupid girl hahah do not be negative i want it too but i have to wait we can be tortured together okay,"[you, stupid, girl, hahah, do, not, be, negative, i, want, it, too, but, i, have, to, wait, we, can, be, tortured, together, okay]"
6543,more snow this morning here in the adirondacks it s covering my crocus that were coming up,"[more, snow, this, morning, here, in, the, adirondacks, it, s, covering, my, crocus, that, were, coming, up]"


In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data['normalized_tweet'][0]

['not',
 'yet',
 'unfortunately',
 'another',
 'few',
 'week',
 'i',
 've',
 'been',
 'told',
 'how',
 'are',
 'you',
 'i',
 've',
 'been',
 'a',
 'little',
 'bit',
 'busy',
 'with',
 'my',
 'latest',
 'project']

In [ ]:
train = data[0:25600]
validation = data[25600:28800]
test = data[28800:]
train.head()

,Unnamed: 0,text,hashtag,emojis,sentiment,normalized_tweet
0,328,not yet unfortunately another few weeks i ve been told how are you i ve been a little bit busy with my latest project,[],[],0,"[not, yet, unfortunately, another, few, week, i, ve, been, told, how, are, you, i, ve, been, a, little, bit, busy, with, my, latest, project]"
1,736,still doing my homework,[],[],0,"[still, doing, my, homework]"
2,1587841,you got an course like that in your degree never heard of it,[],[],1,"[you, got, an, course, like, that, in, your, degree, never, heard, of, it]"
3,1592409,you stupid girl hahah do not be negative i want it too but i have to wait we can be tortured together okay,[],[],1,"[you, stupid, girl, hahah, do, not, be, negative, i, want, it, too, but, i, have, to, wait, we, can, be, tortured, together, okay]"
4,6543,more snow this morning here in the adirondacks it s covering my crocus that were coming up,[],[],0,"[more, snow, this, morning, here, in, the, adirondacks, it, s, covering, my, crocus, that, were, coming, up]"


In [ ]:
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument

In [ ]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=r['normalized_tweet'], tags=r['sentiment']), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=r['normalized_tweet'], tags=r['sentiment']), axis=1)

In [ ]:
train_tagged.values[30]

TaggedDocument(words=['i', 'guess', 'i', 'm', 'out', 'of', 'funny'], tags='0')

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
from tqdm import tqdm

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 25600/25600 [00:00<00:00, 2170052.19it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 25600/25600 [00:00<00:00, 2696014.82it/s]


CPU times: user 1min 7s, sys: 7.95 s, total: 1min 14s
Wall time: 46.2 s


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5678125
Testing F1 score: 0.5677840104518899
